### **Part 4**
**Tasks**
* explore proper visual stimuli
  * spontaneous state
  * (done) evoked state: drifting
* Communication subspaces across different areas
  * (done) correlation
  * (done) dimensionality
    * intra_areas
    * inter_areas
* Information flow across subspaces
  * (done) direction: population correlation as the function of the time delay between areas ($t_2$-$t_1$)
  * intensity (correlation intensity)
* Hierarchy of information flow  across these subspaces
  * direction
    * (done) inter_areas cca_delay
    * *intra_areas cca_delay (layer_dependent)*
    * (done) feedfward and feedback
    * (done) input/output dimensions
  * function validation
* Relation between different information flow channels
  * *subspace & layer_dependent*
  * shared dimensionility
  * (done) generalize canonical dimension across subspaces
  * (done) subspaces angle
  * CCA across subspaces
  * predictive coding
* (to do) State dependent geometrical properties of these channels
* (to do) Dynamical system across areas modeled by multi-region RNN network

*****************
(doing) 
* clear up code, make them beauty
* speed up code, make them faster
* generize code across areas
****************

### Task7 Layer dependent information flow

### Task8 Relation between area_subspace & layer_channel